In [1]:
%load_ext jupyter_black

## Load Packages <a class="anchor" id="load_packages"></a>

Packages relavant to th code are loaded below. Additionally, specific options or parameters are also set. For example, display options are set for `pandas` and the environment path to the environment variable file is provided for the `dotenv` package.

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import sys
import pandas as pd
import numpy as np
import requests
from spotify_dl import spotify_dl
from pathlib import Path
import time
import os
from dotenv import load_dotenv  # changed magic command to explicit load
import librosa
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import pairwise
from sklearn.model_selection import train_test_split
from typing import List
from flask import Flask, redirect, request

pd.set_option("display.max_rows", None)  # pandas dataframe formatting options
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format


custom_env_path = "../../brainstation_capstone_cfg.env"  # environment variables file

## Setting Environment Variables <a class="anchor" id="set_env_vars"></a>

In [3]:
# Spotify Developer Credentials
load_dotenv(dotenv_path=custom_env_path)
CLIENT_ID = os.environ.get("SPOTIPY_CLIENT_ID")
# client ID from app
CLIENT_SECRET = os.environ.get("SPOTIPY_CLIENT_SECRET")
# client secret from app
REDIRECT_URI = os.environ.get("REDIRECT_URI")
# redirect URI - the URI used here matches the one used within the app
SCOPE = "{} {}".format(os.environ.get("SCOPE_PUBLIC"), os.environ.get("SCOPE_PRIVATE"))
# formatted the scope this way to allow for custom configurations in the future
USERNAME = os.environ.get("SPOTIFY_USERNAME")
# Spotify username

[Back to Table of Contents](#toc)

## Feature Function Definitions

In [34]:
# How was this solved? sampling rate must be explicitly passed to every function
# This includes the display function itself!
# Hence, the sampling rate is now an output from the function itself
def get_mfcc(y, sr):
    mfcc = librosa.power_to_db(librosa.feature.mfcc(y=y, sr=sr), ref=np.max)
    return mfcc


def get_melspectrogram(y, sr):
    melspectrogram = librosa.power_to_db(
        librosa.feature.melspectrogram(y=y, sr=sr), ref=np.max
    )
    # this is a power spectrum (amplitude squared)
    return melspectrogram


def get_chroma_vector(y, sr):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return chroma


def get_tonnetz(y, sr):
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    return tonnetz


def get_feature(input_file_path, track_id):
    # Load data
    y, sr = librosa.load(
        input_file_path,
        sr=None,
        offset=10,
        duration=120,
    )
    # Extracting MFCC feature
    mfcc = get_mfcc(y, sr)
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_min = mfcc.min(axis=1)
    mfcc_max = mfcc.max(axis=1)
    mfcc_feature = np.concatenate((mfcc_mean, mfcc_min, mfcc_max))
    np.save(f"../data/vectorized_mp3s/mfcc_{track_id}.npy", mfcc_feature)

    # Extracting Mel Spectrogram feature
    melspectrogram = get_melspectrogram(y, sr)
    melspectrogram_mean = melspectrogram.mean(axis=1)
    melspectrogram_min = melspectrogram.min(axis=1)
    melspectrogram_max = melspectrogram.max(axis=1)
    melspectrogram_feature = np.concatenate(
        (melspectrogram_mean, melspectrogram_min, melspectrogram_max)
    )
    np.save(
        f"../data/vectorized_mp3s/melspectrogram_{track_id}.npy", melspectrogram_feature
    )

    # Extracting chroma vector feature
    chroma = get_chroma_vector(y, sr)
    chroma_mean = chroma.mean(axis=1)
    chroma_min = chroma.min(axis=1)
    chroma_max = chroma.max(axis=1)
    chroma_feature = np.concatenate((chroma_mean, chroma_min, chroma_max))
    np.save(f"../data/vectorized_mp3s/chroma_{track_id}.npy", chroma_feature)

    # Extracting tonnetz feature
    tntz = get_tonnetz(y, sr)
    tntz_mean = tntz.mean(axis=1)
    tntz_min = tntz.min(axis=1)
    tntz_max = tntz.max(axis=1)
    tntz_feature = np.concatenate((tntz_mean, tntz_min, tntz_max))
    np.save(f"../data/vectorized_mp3s/tonnetz_{track_id}.npy", tntz_feature)

    return chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature
    # feature = np.concatenate(
    #     (chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature)
    # )
    # return feature


# May not have use of this function yet....need to fix pairwise comparison
# Save spectra one stage before as separate vectors BEFORE concatenation
def create_spectra(spec_data, sr, type, track_id):
    if type == "mfcc":
        plt.figure()
        librosa.display.specshow(mel_data, y_axis="mel", x_axis="time", sr=sr)
        plt.title("Mel Frequency Cepstral Coefficients")
        plt.colorbar(format="%+2.0f dB")
        plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
    if type == "melspectrogram":
        plt.figure()
        librosa.display.specshow(mel_data, y_axis="mel", x_axis="time", sr=sr)
        plt.title("Melspectrogram")
        plt.colorbar(format="%+2.0f dB")
        plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
    if type == "chroma":
        plt.figure()
        librosa.display.specshow(
            mel_data, y_axis="chroma", x_axis="time", sr=sr, vmin=0, vmax=1
        )
        plt.title("Chroma STFT")
        plt.colorbar(format="%+2.0f dB")
        plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
    if type == "tonnetz":
        plt.figure()
        librosa.display.specshow(
            mel_data, y_axis="tonnetz", x_axis="time", sr=sr, vmin=0, vmax=1
        )
        plt.title("Tonnetz")
        plt.colorbar(format="%+2.0f dB")
        plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
        # Will have to get the track_id to save the file
        # Not sure if the spectra must have the axis labels or the title

In [35]:
test_file = "/Users/vii/repos/brainstation_capstone/data/mp3s/1ZB2qWsheGabSEYvBYxjKn/Take on Me/Weezer - Take on Me.mp3"
track_id = "1ZB2qWsheGabSEYvBYxjKn"
chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature = get_feature(
    test_file, track_id
)

In [41]:
chroma_feature

array([4.8079693e-01, 4.7332811e-01, 4.6357048e-01, 5.2806771e-01,
       4.2110389e-01, 4.6383226e-01, 4.3805096e-01, 5.2115756e-01,
       5.3556848e-01, 4.2572179e-01, 4.7811046e-01, 4.3348038e-01,
       1.2953511e-03, 8.6728914e-04, 3.4206294e-04, 4.9556966e-04,
       6.2709115e-04, 8.0626103e-04, 3.5152116e-03, 4.4452250e-03,
       6.3069914e-03, 3.7467566e-03, 3.2613403e-03, 1.3946538e-03,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00],
      dtype=float32)

In [48]:
chroma_feature.shape

(36,)

In [42]:
chroma_npy = np.load("../data/vectorized_mp3s/chroma_1ZB2qWsheGabSEYvBYxjKn.npy")
chroma_npy

array([4.8079693e-01, 4.7332811e-01, 4.6357048e-01, 5.2806771e-01,
       4.2110389e-01, 4.6383226e-01, 4.3805096e-01, 5.2115756e-01,
       5.3556848e-01, 4.2572179e-01, 4.7811046e-01, 4.3348038e-01,
       1.2953511e-03, 8.6728914e-04, 3.4206294e-04, 4.9556966e-04,
       6.2709115e-04, 8.0626103e-04, 3.5152116e-03, 4.4452250e-03,
       6.3069914e-03, 3.7467566e-03, 3.2613403e-03, 1.3946538e-03,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00],
      dtype=float32)

In [43]:
x = np.array([1, 2])

In [44]:
np.array_equal(
    chroma_feature, chroma_npy
)  # test if the two arrays are the same after saving and loading

True

In [47]:
np.array_equal(chroma_npy, x)  # sanity check

False

## Vectorize Data

In [ ]:
data_dir = Path('../data/mp3s/')
output_dir = Path('../data/vectorized_mp3s/')
path_glob = data_dir.rglob('*.mp3')
file_paths = []
for file_path in path_glob:
    file_paths.append(file_path) # creates a list for repeated iteration
    # if this is not done, the .rglob command above has to be repeated to regenerate iterator
len(file_paths) # number of mp3s in directory

In [ ]:
data_dir = Path('../data/mp3s/')
output_dir = Path('../data/vectorized_mp3s/')
downloaded_path_glob = data_dir.rglob('*.mp3')
output_path_glob = output_dir.rglob('*.parquet')
count = 1
file_paths = [file_path for file_path in downloaded_path_glob]
print('Number of MP3 Files: ', len(file_paths),'\n')
vectorized_track_ids = [file_path.stem for file_path in output_path_glob]
for file_path in file_paths:
    print(f'{count}. FILE PATH: \n', f'{file_path}')
    path_split = str(file_path).split('/')
    track_id = path_split[3]
    if (len(vectorized_track_ids)>0) & (track_id in vectorized_track_ids):
        print(f'{track_id} has already been vectorized...skipping...')
        count+=1
    else:
        track = get_feature(file_path)
        vectorized_df = pd.DataFrame(track).T
        vectorized_df['track_id'] = track_id
        vectorized_df = vectorized_df.set_index(vectorized_df.track_id).drop(columns = 'track_id')
        vectorized_df.columns = vectorized_df.columns.astype(str)
        vectorized_df.to_parquet(f"../data/vectorized_mp3s/{track_id}.parquet")
        count+=1